In [1]:
!pip install mlxtend

In [2]:
import pandas as pd
import numpy as np
import pyodbc
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import math
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

import psycopg2
import pandas.io.sql as psql
from tqdm.autonotebook import tqdm

C:\Users\NaAmornrat\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\NaAmornrat\AppData\Local\Continuum\anaconda3\lib\site-packages\tqdm\autonotebook\__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
# #Connecting to SQL
# driver = 'SQL Server'
# server = '10.0.21.97'
# database = 'VPM_DATA'
# username = 'VPM_Satthapat'
# password = 'Satthapat_VPM'
# cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')

In [4]:
# AND (
# FAST_FASHION_FLAG = 0
# AND EVERYDAY_BASIC_FLAG = 0
# AND HI_END_THAI_DESIGNERS_FLAG = 1
# AND EVENING_GOWN_FLAG = 0
# AND SPORTS_LOOK_FLAG = 0
# AND ADVENTURE_FLAG = 0
# AND WORKING_BUSINESS_FLAG = 0
# AND SUPER_LUXURY_AND_LUXURY_FLAG = 0
# AND WEDDING_GOWN_FLAG = 0
# AND CASUAL_STREET_JEANS_FLAG = 0
# AND KHUN9_FLAG = 0
# AND YOUNG_SASSY_GIRLY_FLAG = 0
# )

In [6]:
connection = psycopg2.connect(user="amornrat",
                                password="Pulse1234",
                                host="the1-rds-aurora-1-instance-1.cijy1qoqmhdq.ap-southeast-1.rds.amazonaws.com",
                                port="5432",
                                database="postgres")

###Prepare Data#####
dset = {}
seg = ['All']
for segment in seg:
    sql = ("""
                select customer_id,cleaned_brandname
                from vpm_data.a_beautycounter_spending_by_cust_brand 
                where customer_id <> ''
                order by customer_id
               """) 

    cursor = connection.cursor()
    data = psql.read_sql(sql, connection)
#     dset.drop(unused_col,1,inplace=True)
    cursor.close()
    print('finish seg: %s %i records' % (segment,len(dset)))
    data.to_pickle('dset_%s.pickle' %segment)
    

finish seg: All 0 records


In [8]:
data = pd.read_pickle('dset_All.pickle')

In [9]:
data

,customer_id,cleaned_brandname
0,000015DC-FA3C-4A3E-9D6A-2235E06DBEA6,BIOTHERM
1,000015DC-FA3C-4A3E-9D6A-2235E06DBEA6,KIEHLS
2,000015DC-FA3C-4A3E-9D6A-2235E06DBEA6,ORIGINS
3,0000242A-6989-4735-A5CB-B226F9D1B6FA,CHRISTIAN DIOR
4,00003551-9E2A-414B-9FAD-00B971EE57DC,CHRISTIAN DIOR
...,...,...
1336968,FFFFD033-3BC0-42EB-BDFE-2B1D6B6DA72A,CLINIQUE
1336969,FFFFE496-E09F-4E1A-83F8-4B4BE42B5C6C,GUCCI
1336970,FFFFE577-949B-4F56-AC6E-FDAAE29AA640,MAC
1336971,FFFFF38F-FFE4-4188-92FB-E77F919D0A60,ESTEE LAUDER


In [10]:
#prep data
def asso_rule_fn(data,column_id,interested_column):
    asso_data=[]
    temp=[]
    start_data=data.loc[0,column_id]
    for i in range(len(data)):
        current_data=data.loc[i,column_id]
        if start_data==current_data:
            temp.append(data.loc[i,interested_column])
        else:
            start_data=current_data
            asso_data.append(temp)
            temp=[]
            temp.append(data.loc[i,interested_column])
    return asso_data

In [11]:
data_for_asso = asso_rule_fn(data,'customer_id','cleaned_brandname')

In [12]:
pd.DataFrame(data_for_asso)

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,BIOTHERM,KIEHLS,ORIGINS,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,CHRISTIAN DIOR,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,CHRISTIAN DIOR,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,LACOSTE,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,CLINIQUE,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722330,AESOP,BENEFIT,BIOTHERM,BOBBI BROWN,CHANEL,CHRISTIAN DIOR,LA MER,SKII,None,None,...,None,None,None,None,None,None,None,None,None,None
722331,CLINIQUE,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
722332,GUCCI,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
722333,MAC,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [13]:
te = TransactionEncoder()
te_ary = te.fit(data_for_asso).transform(data_for_asso)

In [14]:
df = pd.DataFrame(te_ary, columns=te.columns_)

In [15]:
df

,ACSEINE,AESOP,ANNA SUI,ARAMIS,ARMANI,AVEDA,AZZARO,BENEFIT,BIOTHERM,BOBBI BROWN,...,THREE,TOM FORD BEAUTY,TORY BURCH,URBAN DECAY,VALMONT,VERSACE,VIKTOR & ROLF,WHOO,YSL,YVES SAINT LAURENT
0,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722330,False,True,False,False,False,False,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
722331,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
722332,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
722333,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [16]:
df.to_pickle('data_df.pickle')

In [17]:
df = pd.read_pickle('data_df.pickle')

In [27]:
data['customer_id']

0          000015DC-FA3C-4A3E-9D6A-2235E06DBEA6
1          000015DC-FA3C-4A3E-9D6A-2235E06DBEA6
2          000015DC-FA3C-4A3E-9D6A-2235E06DBEA6
3          0000242A-6989-4735-A5CB-B226F9D1B6FA
4          00003551-9E2A-414B-9FAD-00B971EE57DC
                           ...                 
1336968    FFFFD033-3BC0-42EB-BDFE-2B1D6B6DA72A
1336969    FFFFE496-E09F-4E1A-83F8-4B4BE42B5C6C
1336970    FFFFE577-949B-4F56-AC6E-FDAAE29AA640
1336971    FFFFF38F-FFE4-4188-92FB-E77F919D0A60
1336972    FFFFFBC8-2061-46C1-BA15-A4F46ADD3E70
Name: customer_id, Length: 1336973, dtype: object

In [32]:
used = set()
mylist = data['customer_id']
unique = [x for x in mylist if x not in used and (used.add(x) or True)]

In [33]:
unique

['000015DC-FA3C-4A3E-9D6A-2235E06DBEA6',
 '0000242A-6989-4735-A5CB-B226F9D1B6FA',
 '00003551-9E2A-414B-9FAD-00B971EE57DC',
 '00004015-E56B-432B-BCE8-0C6B77982058',
 '0000612B-62FB-4C00-B28D-9E4161FBA80E',
 '000075D2-9D1D-4DB5-9D02-E9248111D65B',
 '00009E62-4CD3-414C-9EEB-7C45E301E0B9',
 '0000D269-851E-4A81-96B6-E312920FAB7B',
 '0000E3AE-114C-4407-A4C3-826798246C69',
 '0000E9F9-1C00-4664-8827-5A62CFEB2FCC',
 '0000F4FB-D55E-4BB6-8898-78F8AD001BD3',
 '00010CF1-B946-4570-9214-B4B1B1BA8189',
 '0001209E-8C19-4831-882A-5433D7863017',
 '000178FF-195A-4512-845E-E043CA6B63EF',
 '0001EC72-78A0-438D-9870-9E8A0830894B',
 '00020260-137A-4A63-B856-E68FBE2FC3AE',
 '0002085E-D04B-4A27-9F2B-5C6FB53BA8D0',
 '00022597-6754-48FD-8E80-6A3196DADCBE',
 '00023FFB-7768-4E4D-9FCF-01C0AA10296D',
 '00024398-0D15-4C49-A66A-BFBF49E621E6',
 '000244BA-D645-4407-9149-30EF731D3CC6',
 '0002483C-5FBB-4075-B555-87964523CBF8',
 '00025DCD-004C-4820-A453-BB3516A550A9',
 '000274E4-4DB3-4F38-BBBE-D92E588938FA',
 '00027538-A3A4-

In [37]:
data_prep = pd.concat([pd.DataFrame(unique),df], axis = 1)

In [38]:
data_prep.to_csv('data_prep.csv')

In [39]:
data_prep

,0,ACSEINE,AESOP,ANNA SUI,ARAMIS,ARMANI,AVEDA,AZZARO,BENEFIT,BIOTHERM,...,THREE,TOM FORD BEAUTY,TORY BURCH,URBAN DECAY,VALMONT,VERSACE,VIKTOR & ROLF,WHOO,YSL,YVES SAINT LAURENT
0,000015DC-FA3C-4A3E-9D6A-2235E06DBEA6,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,0000242A-6989-4735-A5CB-B226F9D1B6FA,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,00003551-9E2A-414B-9FAD-00B971EE57DC,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,00004015-E56B-432B-BCE8-0C6B77982058,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,0000612B-62FB-4C00-B28D-9E4161FBA80E,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722331,FFFFD033-3BC0-42EB-BDFE-2B1D6B6DA72A,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
722332,FFFFE496-E09F-4E1A-83F8-4B4BE42B5C6C,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
722333,FFFFE577-949B-4F56-AC6E-FDAAE29AA640,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
722334,FFFFF38F-FFE4-4188-92FB-E77F919D0A60,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [85]:
frequent_itemsets = apriori(df, min_support=0.0001, use_colnames=True, low_memory = True, max_len = 2)

In [86]:
frequent_itemsets

,support,itemsets
0,0.000800,(ACSEINE)
1,0.011365,(AESOP)
2,0.000372,(ANNA SUI)
3,0.022227,(ARMANI)
4,0.008680,(AVEDA)
...,...,...
1607,0.000386,"(TOM FORD BEAUTY, WHOO)"
1608,0.002286,"(TOM FORD BEAUTY, YVES SAINT LAURENT)"
1609,0.000541,"(YVES SAINT LAURENT, URBAN DECAY)"
1610,0.000489,"(YVES SAINT LAURENT, VERSACE)"


In [70]:
#frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

In [71]:
#asso_rule=association_rules(frequent_itemsets,min_threshold=0.0001)

In [87]:
asso_rule=association_rules(frequent_itemsets,metric = 'lift')

In [88]:
asso_rule

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(BOBBI BROWN),(ACSEINE),0.113187,0.000800,0.000161,0.001419,1.773100,0.000070,1.000620
1,(ACSEINE),(BOBBI BROWN),0.000800,0.113187,0.000161,0.200692,1.773100,0.000070,1.109476
2,(CHANEL),(ACSEINE),0.109185,0.000800,0.000137,0.001255,1.568719,0.000050,1.000456
3,(ACSEINE),(CHANEL),0.000800,0.109185,0.000137,0.171280,1.568719,0.000050,1.074929
4,(CHRISTIAN DIOR),(ACSEINE),0.141661,0.000800,0.000134,0.000948,1.184656,0.000021,1.000148
...,...,...,...,...,...,...,...,...,...
2603,(URBAN DECAY),(YVES SAINT LAURENT),0.004527,0.038767,0.000541,0.119572,3.084346,0.000366,1.091779
2604,(YVES SAINT LAURENT),(VERSACE),0.038767,0.009136,0.000489,0.012606,1.379846,0.000135,1.003514
2605,(VERSACE),(YVES SAINT LAURENT),0.009136,0.038767,0.000489,0.053493,1.379846,0.000135,1.015558
2606,(WHOO),(YVES SAINT LAURENT),0.004674,0.038767,0.000687,0.146919,3.789774,0.000505,1.126778


In [89]:
asso_rule.to_csv('Result_Beauty_LaMer.csv')